In [1]:
# lo_polygon = ox.geocode_to_gdf('R176095', by_osmid=True).to_crs(epsg=32636)
# spb_polygon = ox.geocode_to_gdf('R337422', by_osmid=True).to_crs(epsg=32636).buffer(3000)
# city = lo_polygon.union(spb_polygon).to_crs(epsg=4326) #  get lo polygon
# polygon = gpd.GeoDataFrame([{'name': 'Ленинградская область', 'geometry': city[0]}], crs="EPSG:4326")

In [2]:
# gg= he.convert_list_attr_to_str(g.graph)
# for node,e,data in gg.edges(data=True):
#     if 'geometry' in data:
#         data['geometry']=str(data['geometry'])
# nx.write_graphml(gg, f"tula.graphml")

In [1]:
# Essential imports

from transport_frames.graphbuilder.graph import Graph 
from transport_frames.frame_grader.advanced_grade import AdvancedGrader
from transport_frames.framebuilder.frame import Frame
import transport_frames.utils.helper_funcs as he
from transport_frames.indicators.indicator_terr import indicator_territory
from transport_frames.indicators.indicator_area import indicator_area
from transport_frames.indicators.utils import availability_matrix, create_service_dict
import osmnx as ox
import geopandas as gpd
import networkx as nx
from shapely import Polygon, MultiPolygon
import momepy
import shapely
from dongraphio import GraphType
from iduedu import get_adj_matrix_gdf_to_gdf
from iduedu import get_intermodal_graph
from transport_frames.utils.helper_funcs import prepare_graph


/Users/sashamorozov/Documents/Code/NCCR/transport_frames/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# CRS (в формате ESPG) и OSM ID (id полигона региона, id точки админ. центра региона) по тестовым регионам

# Ленинградская_область EPSG: 32636  / 176095 / N27025179 (Гатчина)
# Санкт_Петербург EPSG:32636 / 337422 / N27490597
# Москва EPSG:32637 / 102269 / N1686293227
# Волгоградская_область EPSG:32638 / 77665 /  N27504363
# Тульская_область EPSG:32637 /  81993 / N34389350
# Омская_область EPSG:32643 / 140292 /  N27503946
# Краснодарский_край EPSG:32637 / 108082 / N27505129
# Тюменская_область EPSG:32642 / 140291 / N27505666
# Московская_область EPSG:32637 / 51490 / N1686293227

## Graph creation

In [ ]:
# Obtaining the region geometry 

region_name = '...'
geocode = ...
local_crs = ...

polygon = ox.geocode_to_gdf(f'R{geocode}',by_osmid=True)
polygon['name'] = region_name

In [2]:
# Obtaining the region geometry (from 2 polygons)

region_name = 'Ленинградская область'
geocode_LO = 176095
geocode_SPb = 337422
local_crs = 32636

region_polygon = ox.geocode_to_gdf(f'R{geocode_LO}', by_osmid=True).to_crs(epsg=local_crs)
city_polygon = ox.geocode_to_gdf(f'R{geocode_SPb}', by_osmid=True).to_crs(epsg=local_crs).buffer(3000)
polygon = gpd.GeoDataFrame(
    region_polygon.union(city_polygon).to_crs(epsg=4326),
    geometry=region_polygon.union(city_polygon).to_crs(epsg=4326),
    crs = 4326
)
polygon['name'] = region_name
polygon.explore()

In [3]:
# Drive graph construction (from polygon)

g = Graph.from_polygon(polygon, crs=local_crs)

2024-09-19 14:03:15.141 | INFO     | transport_frames.graphbuilder.graph:from_polygon:69 - Downloading the graph from OSM...
2024-09-19 14:04:17.517 | INFO     | transport_frames.graphbuilder.graph:_prepare_attrs:135 - Preparing the graph...
2024-09-19 14:06:08.554 | INFO     | transport_frames.graphbuilder.graph:_prepare_attrs:173 - Graph is ready!


In [4]:
# g = Graph.from_polygon(polygon, crs=local_crs)

tula = he.convert_geometry_from_wkt(nx.read_graphml('/Users/polina/Desktop/github/transport_frames/transport_frames/tula.graphml'))
tula = he.convert_list_attr_from_str(tula)
g =  Graph(tula,crs=local_crs)
g.graph.graph['crs'] = 32637

In [4]:
# Reading intermodal (public transport) graph

lo = he.convert_geometry_from_wkt(nx.read_graphml('/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/graphs/Ленинградская_область/Ленинградская_область_inter.graphml'))
lo = he.convert_list_attr_from_str(lo)
lo = prepare_graph(lo)

lo.graph['crs'] = 32636

In [15]:
# n,e = momepy.nx_to_gdf(lo)
# e.explore()
# e[e['type']!='walk'].explore()

## Frame creation

In [5]:
# Add data
regions = gpd.read_file('/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/regions_of_russia.geojson')
regions = regions[regions['ISO3166-2']!='RU-CHU']
regions = regions[regions['geometry'].apply(lambda x: isinstance(x, (Polygon, MultiPolygon)))]

admin_centers = gpd.read_file('/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/Ленинградская_область/Ленинградская_область_district_admin_centres_18_nodes.geojson')
admin_centers = admin_centers.rename(columns={'name188': 'name'})
admin_centers = admin_centers[['name','geometry']].reset_index()
# admin_centers = admin_centers[admin_centers.is_admin_centre_district==1][['name','geometry']].reset_index() 



In [9]:
admin_centers

,index,name,geometry
0,0,Бокситогорск,POINT (33.84600 59.47121)
1,1,Кингисепп,POINT (28.60001 59.37898)
2,2,Кириши,POINT (32.00965 59.45058)
3,3,Кировск,POINT (30.98178 59.87539)
4,4,Тихвин,POINT (33.54145 59.64413)
5,5,Лодейное Поле,POINT (33.54335 60.73432)
6,6,Гатчина,POINT (30.12230 59.56906)
7,7,Подпорожье,POINT (34.16488 60.91260)
8,8,Приозерск,POINT (30.11283 61.05036)
9,9,Сосновый бор,POINT (29.09199 59.90451)


In [6]:
# Transport frame construction from drive graph

f = Frame(g.graph, regions, polygon, admin_centers)
# f.get_geopackage()

## Grading custom territory (frame-based)

In [7]:
# Custom territory data

custom_terr = gpd.read_file('/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/Ленинградская_область/Ленинградская_область_settlement.geojson').iloc[[4]]
custom_terr

,fid,name,layer,2019,2021,2022,2023,2020,status,geometry
4,1,Бокситогорское городское поселение,Бокситогорский муниципальный район,15370,15088,14606,15960,15200,settlement,"MULTIPOLYGON (((34.06836 59.40315, 34.05535 59..."


In [8]:
# Grading custom territory

graded_terr = f.grade_territory(custom_terr, include_priority=True)

In [9]:
graded_terr

,name,layer,status,geometry,grade
4,Бокситогорское городское поселение,Бокситогорский муниципальный район,settlement,"MULTIPOLYGON (((560659.317 6585430.874, 559919...",5


## Criteria of the territory


In [10]:
settlement_points = gpd.read_file('/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/Ленинградская_область/Ленинградская_область_region_points.geojson')
settlement_polygons = gpd.read_file('/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/Ленинградская_область/Ленинградская_область_settlement.geojson')
railway_stations = gpd.read_file('/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/Ленинградская_область/Ленинградская_область_railway_stations.geojson')
bus_stops = gpd.read_file('/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/Ленинградская_область/Ленинградская_область_bus_stops.geojson')
aero = gpd.read_file('/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/Ленинградская_область/Ленинградская_область_local_aerodrome.geojson')

In [ ]:
# inter = get_intermodal_graph(osm_id=geocode,
#                         keep_routes_geom=True, clip_by_bounds=True)

In [31]:
adj_drive = availability_matrix(g.graph, settlement_points,settlement_points,local_crs=local_crs)
adj_inter = availability_matrix(lo, settlement_points, settlement_points, local_crs=local_crs)

In [79]:
grader = AdvancedGrader(local_crs)
cri = grader.get_criteria(graded_terr=graded_terr,
                          points=settlement_points,
                          polygons=settlement_polygons,
                          r_stops=railway_stations,
                          b_stops=bus_stops,
                          aero=aero,
                          adj_mx_drive=adj_drive,
                          adj_mx_inter=adj_inter)


/Users/sashamorozov/Documents/Code/NCCR/transport_frames/.venv/lib/python3.10/site-packages/numpy/core/_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/Users/sashamorozov/Documents/Code/NCCR/transport_frames/.venv/lib/python3.10/site-packages/numpy/core/_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


KeyError: 'name_left'

In [13]:
grader.interpret_gdf()

[('Белев',
  ['Территория расположена в непосредственной близости от одной из региональной трассы (в радиус 5 км от границ территории попадает хотя бы 1 узел региональной трассы, которая является приоритетной, а ближайший федеральный узел находится в не более чем 10км);',
   'Территория попадает в IV квартиль связности (худшие 25% МО) на личном транспорте;',
   'Территория попадает в IV квартиль связности (худшие 25% МО) на общественном транспорте;',
   'В радиусе 15 км отсутствуют порты/причалы/переправы, аэродромы.'])]

## Indicators of the territory (graph-based)

In [46]:
# services
railway_stations = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/Ленинградская_область/Ленинградская_область_railway_stations.geojson')
railway_roads = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/Ленинградская_область/Ленинградская_область_railway_roads.geojson')
bus_stops = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/Ленинградская_область/Ленинградская_область_bus_stops.geojson')
bus_routes = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/Ленинградская_область/Ленинградская_область_bus_routes.geojson')
bus_routes['route'] = bus_routes['desc'] 
fuel_stations = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/Ленинградская_область/Ленинградская_область_fuel_stations.geojson')
local_aerodrome = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/Ленинградская_область/Ленинградская_область_local_aerodrome.geojson')
local_aerodrome['geometry'] = shapely.centroid(local_aerodrome['geometry']).set_crs(local_aerodrome.crs)
# international_aerodrome = local_aerodrome[local_aerodrome['aerodrome:type']=='international']
international_aerodrome = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/Ленинградская_область/Ленинградская_область_international_aerodrome.geojson')
international_aerodrome['geometry'] = shapely.centroid(international_aerodrome['geometry']).set_crs(international_aerodrome.crs)


# nature objects
water = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/Ленинградская_область/Ленинградская_область_water.geojson')
nature_reserve = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/Ленинградская_область/Ленинградская_область_nature_reserve.geojson')

capital =ox.geocode_to_gdf('N27025179',by_osmid=True)
districts_polygons = gpd.read_file('/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/Ленинградская_область/Ленинградская_область_district.geojson')
settlement_polygons = gpd.read_file('/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/Ленинградская_область/Ленинградская_область_settlement.geojson')
settlement_points = gpd.read_file('/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/Ленинградская_область/Ленинградская_область_region_points.geojson')



In [74]:
services = create_service_dict(
                                railway_stations=railway_stations,
                                railway_paths=railway_roads,
                                bus_stops=bus_stops,
                                bus_routes=bus_routes,
                                fuel_stations=fuel_stations,
                                local_aerodrome=local_aerodrome,
                                # international_aerodrome=international_aerodrome,
                                water_objects=water,
                                nature_reserve=nature_reserve,          
                                local_crs=local_crs)

In [78]:
t = indicator_territory(g.graph,custom_terr,services,capital,admin_centers,settlement_points,districts_polygons,local_crs)
t

AttributeError: 'NoneType' object has no attribute 'x'

## Indicators of the area (graph-based)

In [42]:
#area polygons
country_polygon = ox.geocode_to_gdf('RUSSIA')
country_polygon['status'] = 'country'
country_polygon['layer'] = 'country'

In [72]:
services.keys()

dict_keys(['railway_stations', 'fuel_stations', 'ports', 'local_aerodrome', 'international_aerodrome', 'nature_reserve', 'water_objects', 'train_paths', 'bus_stops', 'bus_routes'])

In [76]:
ind_area = indicator_area(g.graph, [settlement_polygons, districts_polygons,country_polygon],settlement_points,services, capital,local_crs,adj_drive,adj_inter)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Processing areas:   0%|          | 0/3 [00:00<?, ?it/s]2024-09-18 18:29:36.228 | INFO     | transport_frames.indicators.indicator_area:indicator_area:77 - Calculating service accessibility
2024-09-18 18:29:36.521 | INFO     | transport_frames.indicators.indicator_area:indicator_area:96 - Calculating distance to region admin center and federal roads
2024-09-18 18:29:39.216 | INFO     | transport_frames.indicators.indicator_area:indicator_area:101 - Calculating train path lengths
2024-09-18 18:29:40.234 | INFO     | transport_frames.indicators.indicator_area:indicator_area:110 - Calculating number of bus routes
2024-09-18 18:29:49.961 | INFO     | transport_frames.indicators.indicator_area:indicator_area:120 - Calculating reg_1 road lengths
2024-09-18 18:29:52.202 | INFO     | transport_frames.indicators.indicator_area:indicator_area:120 - Calculating reg_2 road lengths
2024-09-18 18:29:54.415 | INFO     | transport_frames.indicators.indicator_area:indicator_area:129 - Calculating road d

In [77]:
ind_area[0]


,name,layer,status,geometry,number_of_railway_stations,railway_stations_accessbility_min,number_of_fuel_stations,fuel_stations_accessbility_min,number_of_ports,ports_accessibility_min,...,international_aerodrome_accessibility_min,number_of_bus_stops,connectivity_drive_min,to_region_admin_center_km,to_reg_1_km,train_path_length_km,number_of_bus_routes,reg1_length_km,reg2_length_km,road_density_km/km2
0,Самойловское сельское поселение,Бокситогорский муниципальный район,settlement,"MULTIPOLYGON (((580049.515 6617067.720, 580225...",2.0,18.1770,2.0,20.162,0,None,...,None,68.0,220.9920,300.439,6.088,19.068,12,18.632,18.632,0.113
1,Большедворское сельское поселение,Бокситогорский муниципальный район,settlement,"MULTIPOLYGON (((573514.444 6620389.851, 573579...",8.0,13.1320,1.0,19.651,0,None,...,None,50.0,197.8690,280.318,18.029,23.824,16,32.324,32.324,0.142
2,Пикалевское городское поселение,Бокситогорский муниципальный район,settlement,"MULTIPOLYGON (((564707.328 6594903.594, 564341...",1.0,6.0180,2.0,0.961,0,None,...,None,59.0,204.9890,286.169,0.084,14.023,20,14.456,14.456,1.168
3,Борское сельское поселение,Бокситогорский муниципальный район,settlement,"MULTIPOLYGON (((560629.509 6557332.762, 560642...",0.0,29.0530,1.0,15.473,0,None,...,None,44.0,210.4410,292.402,26.426,0.000,12,0.000,0.000,0.106
4,Бокситогорское городское поселение,Бокситогорский муниципальный район,settlement,"MULTIPOLYGON (((560659.317 6585430.874, 559919...",0.0,13.5520,2.0,5.133,0,None,...,None,34.0,197.3370,286.688,18.846,0.000,15,7.097,7.097,0.281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,Фёдоровское городское поселение,Тосненский муниципальный район,settlement,"MULTIPOLYGON (((360431.648 6619592.469, 360583...",2.0,5.5065,0.0,11.024,0,None,...,None,21.0,111.2180,38.063,7.612,12.336,3,0.000,0.000,1.315
184,Нурминское сельское поселение,Тосненский муниципальный район,settlement,"MULTIPOLYGON (((387941.398 6612286.590, 388009...",2.0,1.2410,0.0,7.613,0,None,...,None,15.0,122.3930,65.189,11.458,9.254,4,0.000,0.000,0.348
185,Красноборское городское поселение,Тосненский муниципальный район,settlement,"MULTIPOLYGON (((363803.737 6614904.469, 363821...",2.0,3.4720,1.0,7.246,0,None,...,None,29.0,111.6755,47.602,3.379,10.498,3,29.583,29.583,0.952
186,Трубникоборское сельское поселение,Тосненский муниципальный район,settlement,"MULTIPOLYGON (((392924.072 6551507.217, 393174...",7.0,8.4705,4.0,7.198,0,None,...,None,44.0,146.3060,100.131,8.596,17.975,4,77.274,77.274,0.198
